In [16]:
import os
from ipynb.fs.full.preprocessing import parse_observations
import re

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np

In [4]:
def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

def sample_sentence(hmm, obs_map, n_words=100):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)

    # Sample and convert sentence.
    emission, states = hmm.generate_emission(n_words)
    sentence = [obs_map_r[i] for i in emission]

    return ' '.join(sentence).capitalize()

def generate_obs(hmm, obs_map):
    '''
    Naively generates 14-line sonnet with 10 words each.
    
    Inputs:
    hmm: trained hmm
    obs_map: maps word to observation index
    
    Outputs:
    None
    '''
    # generate all words in sonnet
    all_words = sample_sentence(hmm, obs_map, 140)
    
    # split into 14 lines and add capitalization/naive punctuation
    for i in range(14):
        count = 0
        line = ' '.join(all_words[i*10:(i+1)*10]).capitalize()
        if i == 11 or i == 12:
            line += ','
        else:
            line += '.'
        print(line)

In [31]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_map = dict((c, i) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [34]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_map[char] for char in seq_in])
    dataY.append(char_map[seq_out])
    
n_sequences = len(dataX)

In [38]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [39]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# train the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
68736/97635 [====================>.........] - ETA: 33s - loss: 2.9730